In [19]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds

In [33]:
class CustomPoolLayer(layers.Layer):
    
    def __init__(self, k=2, padding='SAME'):#padding='VALID'):
        super(CustomPoolLayer, self).__init__()
        self.k = k
        self.p = padding
    
    def call(self, inputs):
        return tf.nn.max_pool2d(inputs, ksize=[1, self.k, self.k,1], strides=[1, self.k, self.k, 1], padding=self.p)

class CustomConvLayer(layers.Layer):

    def __init__(self, shape, bias=True, stride=1, padding='SAME'):

        super(CustomConvLayer, self).__init__()
        self.bias = bias
        self.w = self.add_weight(
            shape=shape,
            initializer='glorot_uniform',
            trainable=True,
            name='w'
        )
        self.m = self.add_weight(
            shape=shape,
            initializer='ones',
            trainable=False,
            name='m'
        )
        if self.bias==True:
            self.b = self.add_weight(
                shape=shape[-1],
                initializer='zeros',
                trainable=True,
                name='b'
            )
        self.s = stride
        self.p = padding
        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p,)
        if self.bias == True:
            x = tf.nn.bias_add(x, self.b)
        
        return tf.nn.relu(x)

class CustomDenseLayer(layers.Layer):
    def __init__(self, shape, bias, activation = 'relu'):
        super(CustomDenseLayer, self).__init__()
        self.bias = bias
        self.w = self.add_weight(
            shape = shape,
            initializer='glorot_uniform',
            trainable = True,
            name='w'
        )
        self.m = self.add_weight(
            shape = shape,
            initializer='ones',
            trainable = False,
            name='m'
        )
        if self.bias == True:
            self.b = self.add_weight(
                shape = (shape[-1]),
                initializer = 'zeros',
                trainable = True,
                name='b'
            )
        self.a = activation
        
        
    def call(self, inputs):
        x = tf.matmul(inputs, tf.multiply(self.w, self.m))
        if self.bias == True:
            x = tf.nn.bias_add(x, self.b)
        if self.a == 'relu':
            return tf.nn.tanh(x)
        if self.a == 'softmax':
            return tf.nn.softmax(x)
        
cnn_shapes = {
    # 5x5 conv, 1 input, 6 outputs
    'conv_1': (5, 5, 1, 6),
    # 5x5 conv, 6 inputs, 16 outputs
    'conv_2': (5, 5, 6, 16),
    #5x5 conv as in paper, 16 inputs, 120 outputs
    'conv_3': (1, 1, 16, 120),
    # fully connected, 5*5*16 inputs, 120 outputs
    'dense_1': (4*4*16, 120),
    # fully connected, 120 inputs, 84 outputs
    'dense_2': (120, 84),
    # 84 inputs, 10 outputs (class prediction)
    'dense_3': (84, 10),
}
    
class CNN(tf.keras.Model):
    def model(self):
        x = Input(shape=(28*28))
        return Model(inputs=[x], outputs=self.call(x))

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = CustomConvLayer(cnn_shapes['conv_1'], True, 1, 'VALID')#'VALID')
        self.maxpool1 = CustomPoolLayer(k=2, padding='SAME')
        self.conv2 = CustomConvLayer(cnn_shapes['conv_2'], True, 1, 'VALID')
        self.maxpool2 = CustomPoolLayer(k=2, padding='VALID')

        self.dense1 = CustomDenseLayer(cnn_shapes['dense_1'], True, 'relu')
        self.dense2 = CustomDenseLayer(cnn_shapes['dense_2'], True, 'relu')
        self.dense3 = CustomDenseLayer(cnn_shapes['dense_3'], True, 'softmax')
        #self.conv_layers = []
        #self.conv_masks = []
        #self.dense_layers = []
        #self.dense_masks = []
        self.conv_layers = [0, 3]
        self.conv_masks = [2, 5]
        self.dense_layers = [6, 9, 12]
        self.dense_masks = [8, 11, 14]
        
        
    def call(self, inputs):
        print(inputs.shape)
        x = tf.reshape(inputs, shape=[-1,28, 28, 1])
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.maxpool1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.maxpool2(x)
        print(x.shape)
        x = layers.Flatten()(x)
        print(x.shape)
        x = self.dense1(x)
        print(x.shape)
        x = self.dense2(x)
        print(x.shape)
        x = self.dense3(x)
        print(x.shape)
        return x

In [35]:
x = x_train[0]

In [39]:
model = CNN()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy() ,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'],
    experimental_run_tf_function=True
)

In [41]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test),
)

Epoch 1/10
(None, 784, 1)
(None, 28, 28, 1)
(None, 24, 24, 6)
(None, 12, 12, 6)
(None, 8, 8, 16)
(None, 4, 4, 16)
(None, 256)
(None, 120)
(None, 84)
(None, 10)
(None, 784, 1)
(None, 28, 28, 1)
(None, 24, 24, 6)
(None, 12, 12, 6)
(None, 8, 8, 16)
(None, 4, 4, 16)
(None, 256)
(None, 120)
(None, 84)
(None, 10)
468/469 [============================>.] - ETA: 0s - loss: 0.2805 - accuracy: 0.9183(None, 784, 1)
(None, 28, 28, 1)
(None, 24, 24, 6)
(None, 12, 12, 6)
(None, 8, 8, 16)
(None, 4, 4, 16)
(None, 256)
(None, 120)
(None, 84)
(None, 10)
469/469 [==============================] - 15s 32ms/step - loss: 0.2802 - accuracy: 0.9183 - val_loss: 0.0882 - val_accuracy: 0.9733
Epoch 2/10
469/469 [==============================] - 15s 31ms/step - loss: 0.0779 - accuracy: 0.9768 - val_loss: 0.0571 - val_accuracy: 0.9824
Epoch 3/10
469/469 [==============================] - 15s 31ms/step - loss: 0.0586 - accuracy: 0.9823 - val_loss: 0.0459 - val_accuracy: 0.9847
Epoch 4/10
469/469 [=================

In [37]:
model(x)

(784, 1)
(1, 28, 28, 1)
(1, 24, 24, 6)
(1, 12, 12, 6)
(1, 8, 8, 16)
(1, 4, 4, 16)
(1, 256)
(1, 120)
(1, 84)
(1, 10)


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.10321914, 0.09820537, 0.07483614, 0.10841142, 0.1381944 ,
        0.10832212, 0.08782458, 0.13692474, 0.05885972, 0.08520242]],
      dtype=float32)>

In [38]:
dataset = 'MNIST'
ratio = '100%'
ds, info = tfds.load(name=dataset, with_info=True, split=[f"train[:{ratio}]",f"test[:{ratio}]"])
ds_train=ds[0]
ds_test=ds[1]
def load_image(datapoint):
        input_image, label = normalize(datapoint)
        return input_image, label
def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.reshape(y['image'],(28*28,1)), y['label'])
    return y
ds_test = list(ds_test.map(load_image))
ds_train = list(ds_train.map(load_image))

x_train = tf.convert_to_tensor([sample[0] for sample in ds_train])
y_train = tf.convert_to_tensor([sample[1] for sample in ds_train])
x_test = tf.convert_to_tensor([sample[0] for sample in ds_test])
y_test = tf.convert_to_tensor([sample[1] for sample in ds_test])
